# Reference:
* [深入MNIST](http://wiki.jikexueyuan.com/project/tensorflow-zh/tutorials/mnist_pros.html) 
* [實作 MNIST 卷積神經網路 Convolutional Neural Network](https://ithelp.ithome.com.tw/articles/10187149)
* [tf.nn.conv2d是怎样实现卷积的？](https://blog.csdn.net/mao_xiao_feng/article/details/53444333)
* [tf.nn.max_pool实现池化操作](https://blog.csdn.net/mao_xiao_feng/article/details/53453926)
* [Conv Nets: A Modular Perspective](http://colah.github.io/posts/2014-07-Conv-Nets-Modular/)

## 若遇到以下的狀況，可以用 os.environ["TF_CPP_MIN_LOG_LEVEL"]='2' 解決
* Reference: [成功解决Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX AVX2](https://blog.csdn.net/qq_41185868/article/details/79127838) 

```
Warning:
2018-10-25 16:54:30.265307: I tensorflow/core/platform/cpu_feature_guard.cc:137] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
```

In [ ]:
# Just disable the Warning:
# import os
# os.environ["TF_CPP_MIN_LOG_LEVEL"]='2'

# 建構一個多層卷積網路
在 [mnist_softmax_regression.ipynb](./mnist_softmax_regression.ipynb) 上只有 91% 正確率，實在太糟糕。在這個裡，我们用一個稍微複雜的模型：卷積神經網路(CNN: Convolutional Neural Network)來改善效果。這會達到大概 99.2% 的準確率。雖然不是最高，但是還是比較讓人满意的。

In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import datetime

mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

# define placeholder for inputs to network
x = tf.placeholder(tf.float32, [None, 784])  # image data
y_ = tf.placeholder(tf.float32, [None, 10])  # image label

## 權重初始化
為了創建這個模型，我们需要創建大量的權重和偏置項。`這個模型中的權重在初始化時應該加入少量的噪點來打破對稱性以及避免 0 梯度。由於我們使用的是 ReLU activity function，因此比較好的做法是用一個較小的正數來初始化偏置項，以避免神經元節點輸出恒為 0 的問題（dead neurons）`。為了不在建立模型的时候反復做初始化操作，我们定義兩個函數用於初始化。 

In [ ]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

## 卷積(Convolution)和池化(Pooling)
Tensorflow 同樣給我們很大的彈性來做卷積還有池化這兩個動作．如何處理邊界? 我們的 stride 大小要設多少? 在這個範例中，我們會一直使用 vanilla 的版本．我們的卷積過程中的參數 stride 會是 1 而 padded 則是 0．也因此輸入還有輸出都會是同樣的大小 (size)．而我們的 polling 則是用 2X2 的傳統 max polling 來做．為了讓我們的程式更加簡潔，我們同樣把這樣的操作抽象成函數。
* Vanilla 是神經網路領域的常見詞彙，比如 Vanilla Neural Networks、Vanilla CNN 等。Vanilla 本意是香草，在這裡基本等同於 raw。
  * 比如Vanilla CNN 基本上就是最原始的 CNN。 

In [ ]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

## 第一個卷積層
我們現在可以來實現第一個卷積層。他會先有一個卷積接著一個 max polling 來完成。
* 請先了解 tf.nn.conv2d() 的定義。
* 這個卷積會從 5x5 的 patch 算出 32 個特徵, 他的權重 tensor 的形狀是 [5, 5, 1, 32] = [filter_height, filter_width, in_channels, out_channels]。 其表示 patch size = 5x5, 輸入 channel=1 (要與輸入圖片的 in_channel 數相同), 輸出 channel = 32。其實也可以想成 patch size = 5x5x1，共執行了 32 次，所以有 32 個 output.
* 同樣的在輸出也會有偏移量向量 (bias vector)。

In [ ]:
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

現在要來把輸入 x 來導入我們剛剛建立的第一個卷積層，那必須先把 x 轉換成一個 tensor = [batch, in_height, in_width, in_channels] 這樣的shape，具體含義是[訓練時一個 batch 的圖片數量, 圖片高度, 圖片寬度, 圖像通道數]，其中第二個和第三個維度對應到了圖片的寬度和高度，而最後一個則對應到了顏色的 channel 數 (這裡因為是灰階的所以 channel 為 1)
* 請先了解 tf.nn.conv2d() 的定義。

In [ ]:
x_image = tf.reshape(x, [-1, 28, 28, 1])

我們接下來把 x_image 還有權重 tensor 輸入剛剛定義的卷積函數，再來加上偏移值 (bias) 後輸入 ReLU 函數，最後經過 max pooling． max_pool_2x2 函數會把圖片的大小縮成 14x14．
* tf.nn.conv2d(..., padding='SAME') 表示其圖片 size 維持不變，仍為 28x28。在 patch 過程中會超出圖片範圍，超出部分用 0 代替。但其中每個圖片的 output 應該是 28x28x32 (channel=32)。
* tf.nn.max_pool(, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1],) 其表示用 2x2 來做 pooling, 其 stride in heigh/width is 2。所以其圖片 size = 14x14, 其 output = 14x14x32 (channel=32)

In [ ]:
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

## 第二個卷積層
為了建立比較深度的神經網路，我們把許多層疊在一起。第二層會從 5x5 的 patch 中取出 64 個特徵。  
此時圖片 size = 7x7，其 output = 7x7x64 (channel=64)

In [ ]:
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

## 密集的全連接層
現在讓我們想像圖片的大小已經被縮成了 7x7 的大小，我們加入一個 1024 的全連接層來把前面的全部輸出輸入全連接層。其中包含了先把 pooling 的輸出展開後乘上一個權重矩陣再加上一個偏移量向量，最後輸入 ReLU。
* 所以 weight matrix = [7x7x64, 1024]

In [ ]:
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

## Dropout
為了減少 overfitting，在輸出層之前我們會加入 dropout 層．首先我們建立了一個 placeholder 來代表神經元在 dropout 過程中不變的機率．這可以 讓我們決定要在訓練的時候打開 dropout，而在測試的時候關閉 dropout． Tensorflow 的 tf.nn.dropout 除了會遮蔽神經元的輸出以外也會自動對輸入值做 scaling，所以我們在使用的時候可以不用考慮 scale．

註: 對於這種比較小的卷積網路，有沒有 dropout 對於成果不會有太大影響．dropout 是一個非常好的方法來減低 overfitting，但僅限於比較大的神經網路．

In [ ]:
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

## 輸出層
最後我們加上像之前 softmax regression 一樣的層。

In [ ]:
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

## 訓練以及評估模型
那我們的模型表現的如何呢? 這裡我們會用之前 Softmax 範例的大部分程式碼來訓練以及評估這個模型．

不過有幾點不同的是:

* 我們會把 gradient descent 最佳化演算法換成更為精密的 ADAM 最佳化演算法。
* 我們會在 feed_dict 參數之中加入 keep_prob 這個參數來控制 dropout 的機率。

In [ ]:
# Evaluation functions
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Training algorithm
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

# Training steps
time1 = datetime.datetime.now()
with tf.Session() as sess:
    max_steps = 20000
    sess.run(tf.global_variables_initializer())

    for i in range(max_steps):
        batch = mnist.train.next_batch(50)
        sess.run(train_step, feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
        if(i % 1000) == 0:
            print("%05d" % i, sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

    print("%05d" % max_steps, sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))
time2 = datetime.datetime.now()
print("spend time:", time2-time1)

## Output
```
00000 0.1232
01000 0.964
02000 0.9753
03000 0.9809
04000 0.9838
05000 0.987
06000 0.9865
07000 0.9882
08000 0.9906
09000 0.9897
10000 0.9895
11000 0.9917
12000 0.9921
13000 0.9919
14000 0.9915
15000 0.9916
16000 0.9927
17000 0.9927
18000 0.9923
19000 0.9919
20000 0.9916
spend time: 0:01:38.723719
```